# Pima Indians Diabetes Database

[About Dataset](https://www.kaggle.com/datasets/uciml/pima-indians-diabetes-database)
## Context

This dataset is originally from the National Institute of Diabetes and Digestive and Kidney Diseases. The objective of the dataset is to diagnostically predict whether or not a patient has diabetes, based on certain diagnostic measurements included in the dataset. Several constraints were placed on the selection of these instances from a larger database. In particular, all patients here are females at least 21 years old of Pima Indian heritage.

## Content
The datasets consists of several medical predictor variables and one target variable, Outcome. Predictor variables includes the number of pregnancies the patient has had, their BMI, insulin level, age, and so on.

## Acknowledgements
Smith, J.W., Everhart, J.E., Dickson, W.C., Knowler, W.C., & Johannes, R.S. (1988). Using the ADAP learning algorithm to forecast the onset of diabetes mellitus. In Proceedings of the Symposium on Computer Applications and Medical Care (pp. 261--265). IEEE Computer Society Press.

Inspiration
Can you build a machine learning model to accurately predict whether or not the patients in the dataset have diabetes or not?

In [1]:
import random
import numpy as np

random.seed(42)
np.random.seed(100)

In [2]:
import pandas as pd
import os
from f_importance.util.runner import compute_importance

In [3]:
diabetes = pd.read_csv("./dataset/diabetes.csv")

diabetes.sample(10)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
173,1,79,60,42,48,43.5,0.678,23,0
253,0,86,68,32,0,35.8,0.238,25,0
207,5,162,104,0,0,37.7,0.151,52,1
737,8,65,72,23,0,32.0,0.600,42,0
191,9,123,70,44,94,33.1,0.374,40,0
754,8,154,78,32,0,32.4,0.443,45,1
159,17,163,72,41,114,40.9,0.817,47,1
448,0,104,64,37,64,33.6,0.510,22,1
359,1,196,76,36,249,36.5,0.875,29,1
651,1,117,60,23,106,33.8,0.466,27,0


In [4]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [5]:
def train_evaluate( data: pd.DataFrame, clazz= XGBClassifier, target="Outcome", test_size=1/5., proba=False, metrics=metrics.accuracy_score,  **kwargs):
    X_train, X_test, y_train, y_test = train_test_split(data.drop(columns=[target]), data[target], test_size=test_size, random_state=42)
    model: XGBClassifier = clazz(**kwargs)
    model.fit(X_train, y_train)
    if proba:
        y_preds = model.predict_proba(X_test)[:, 1]
    else:
        y_preds = model.predict(X_test)
    score = metrics(y_test, y_preds)
    return score

In [6]:
print("Accuracy Score : ", train_evaluate(diabetes))

Accuracy Score :  0.6883116883116883


In [7]:
importance = compute_importance(
    model_name = "XGBClassifier",
    method = "DataSample",
    metric = "accuracy_score",
    dataset  = diabetes,
    targets = "Outcome",
    n_gram = (1, 1),
    val_rate = 0.15,
    shuffle = True,
    n = 5,
    is_regression = False,
    n_jobs = os.cpu_count(),
    refit = True,
    seed=42
)

TypeError: compute_importance() got an unexpected keyword argument 'seed'

In [8]:
importance

,Importance,Split0,Split1,Split2,Split3,Split4
,0.721739,0.721739,0.721739,0.721739,0.721739,0.721739
'Glucose',0.057391,0.660870,0.730435,0.626087,0.634783,0.669565
'BloodPressure',0.013913,0.721739,0.678261,0.695652,0.730435,0.713043
'DiabetesPedigreeFunction',0.008696,0.686957,0.730435,0.713043,0.721739,0.713043
'Pregnancies',-0.006957,0.747826,0.730435,0.713043,0.721739,0.730435
'Age',-0.010435,0.747826,0.695652,0.747826,0.756522,0.713043
'BMI',-0.024348,0.739130,0.773913,0.739130,0.730435,0.747826
'SkinThickness',-0.026087,0.747826,0.739130,0.739130,0.765217,0.747826
'Insulin',-0.053913,0.791304,0.765217,0.765217,0.765217,0.791304


In [9]:
to_delete = ['SkinThickness', 'BloodPressure']

In [10]:
importance2 = compute_importance(
    model_name = "XGBClassifier",
    method = "DataSample",
    metric = "accuracy_score",
    dataset  = diabetes.drop(columns=[to_delete[0]]),
    targets = "Outcome",
    n_gram = (1, 1),
    val_rate = 0.15,
    shuffle = True,
    n = 5,
    is_regression = False,
    n_jobs = os.cpu_count(),
    refit = True
)

In [11]:
importance2

,Importance,Split0,Split1,Split2,Split3,Split4
,0.808696,0.808696,0.808696,0.808696,0.808696,0.808696
'Glucose',0.259130,0.556522,0.573913,0.478261,0.591304,0.547826
'DiabetesPedigreeFunction',0.113043,0.669565,0.695652,0.704348,0.686957,0.721739
'Pregnancies',0.095652,0.704348,0.695652,0.713043,0.713043,0.739130
'Age',0.081739,0.739130,0.721739,0.730435,0.730435,0.713043
'BMI',0.078261,0.713043,0.747826,0.747826,0.730435,0.713043
'Insulin',0.053913,0.756522,0.730435,0.739130,0.782609,0.765217
'BloodPressure',0.040000,0.773913,0.791304,0.747826,0.756522,0.773913


In [12]:
importance3 = compute_importance(
    model_name = "XGBClassifier",
    method = "DataSample",
    metric = "accuracy_score",
    dataset  = diabetes.drop(columns=[to_delete[1]]),
    targets = "Outcome",
    n_gram = (1, 1),
    val_rate = 0.15,
    shuffle = True,
    n = 5,
    is_regression = False,
    n_jobs = os.cpu_count(),
    refit = True
)

In [13]:
importance3

,Importance,Split0,Split1,Split2,Split3,Split4
,0.721739,0.721739,0.721739,0.721739,0.721739,0.721739
'Glucose',0.126957,0.617391,0.591304,0.591304,0.582609,0.591304
'BMI',0.045217,0.669565,0.678261,0.652174,0.686957,0.695652
'Age',0.005217,0.713043,0.730435,0.713043,0.686957,0.739130
'DiabetesPedigreeFunction',-0.013913,0.713043,0.730435,0.773913,0.704348,0.756522
'Insulin',-0.043478,0.773913,0.747826,0.756522,0.765217,0.782609
'Pregnancies',-0.046957,0.782609,0.773913,0.756522,0.765217,0.765217
'SkinThickness',-0.097391,0.826087,0.808696,0.817391,0.808696,0.834783
